# Comparison: Base GCN vs Edge-Weighted GCN for Illicit Activity Emergence Detection

This notebook compares the performance of two GCN approaches:

1. **Base GCN**: Uses standard binary adjacency (unweighted edges)
2. **Edge-Weighted GCN**: Uses temporal edge weights with the three-step process:
   - Within-step sum (aggregate transactions in same timestep)
   - Exponential decay (emphasize recent transactions)  
   - Temperature-softmax (normalize incoming weights)

We evaluate both models on the same illicit activity emergence prediction task using identical hyperparameters and training procedures to ensure a fair comparison.

## 1. Import Required Libraries and Setup

In [1]:
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add root directory to path for imports
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

# Core libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
import copy

# PyTorch and PyTorch Geometric
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Scikit-learn for metrics
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    accuracy_score, precision_recall_fscore_support
)

# Our graph building functions
from code_lib.graph_builder import (
    build_emergence_graphs_for_time_range,
    compute_temporal_edge_weights_with_defaults,
    extract_node_features,
    extract_node_classes,
    build_edge_index
)
from code_lib.utils import load_parts

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else 
                      "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

Using device: mps


## 2. Load and Prepare Data

In [5]:
# Data paths
DATA_DIR = "../elliptic_dataset"
WALLETS_FEATURES = "wallets_features.csv"
WALLETS_CLASSES = "wallets_classes.csv"
EDGES_PREFIX = "AddrTxAddr_edgelist_part_"

print("Loading Elliptic Dataset...")

# Load node features and classes
print("  Loading wallet features...")
nodes = pd.read_csv(os.path.join(DATA_DIR, WALLETS_FEATURES))
print(f"    Loaded {len(nodes)} node feature records")

print("  Loading wallet classes...")
node_labels = pd.read_csv(os.path.join(DATA_DIR, WALLETS_CLASSES))
print(f"    Loaded {len(node_labels)} node class records")

# Load edge data (aggregated address-to-address transactions)
print("  Loading aggregated edge data...")
edges_with_edge_labels = load_parts(DATA_DIR, EDGES_PREFIX)
print(f"    Loaded {len(edges_with_edge_labels)} edge records")

# Merge node features with class labels
nodes_with_labels = nodes.merge(node_labels, on='address', how='left')
print(f"    Merged dataset has {len(nodes_with_labels)} node records")

# Data overview
print(f"\nDataset Overview:")
print(f"  Time steps: {nodes['Time step'].min()} to {nodes['Time step'].max()}")
print(f"  Unique addresses: {nodes['address'].nunique()}")
print(f"  Edge time steps: {edges_with_edge_labels['Time step'].min()} to {edges_with_edge_labels['Time step'].max()}")
print(f"  Unique edge pairs: {len(edges_with_edge_labels[['input_address', 'output_address']].drop_duplicates())}")

# Class distribution
class_counts = node_labels['class'].value_counts().sort_index()
print(f"\nClass distribution:")
for class_id, count in class_counts.items():
    class_name = {1: 'Illicit', 2: 'Licit', 3: 'Unknown'}[class_id]
    print(f"  {class_name} ({class_id}): {count:,}")

print("\nData loading completed ✓")

Loading Elliptic Dataset...
  Loading wallet features...
    Loaded 1268260 node feature records
  Loading wallet classes...
    Loaded 1268260 node feature records
  Loading wallet classes...
    Loaded 822942 node class records
  Loading aggregated edge data...
    Loaded 822942 node class records
  Loading aggregated edge data...
    Loaded 2868964 edge records
    Loaded 2868964 edge records
    Merged dataset has 1268260 node records

Dataset Overview:
  Time steps: 1 to 49
  Unique addresses: 822942
  Edge time steps: 1 to 49
    Merged dataset has 1268260 node records

Dataset Overview:
  Time steps: 1 to 49
  Unique addresses: 822942
  Edge time steps: 1 to 49
  Unique edge pairs: 2784344

Class distribution:
  Illicit (1): 14,266
  Licit (2): 251,088
  Unknown (3): 557,588

Data loading completed ✓
  Unique edge pairs: 2784344

Class distribution:
  Illicit (1): 14,266
  Licit (2): 251,088
  Unknown (3): 557,588

Data loading completed ✓


## 3. Build Base and Edge-Weighted Graphs

In [7]:
# Graph building parameters
FIRST_TIME_STEP = 1
LAST_TIME_STEP = 49
MAX_WALK_LENGTH = 2
TIME_HORIZON = 3
USE_DISTANCE_LABELS = False  # Binary labels for comparison

# Training/testing split (same as baseline)
TRAIN_START = 1
TRAIN_END = 37
EVAL_START = 40
EVAL_END = 46

# Specific timesteps we need (same as train_gcn_baseline.ipynb)
# Train: indices [19, 22, 29, 30] = timesteps [20, 23, 30, 31]  
# Test: indices [33, 34, 38] = timesteps [34, 35, 39]
REQUIRED_TIMESTEPS = [20, 23, 30, 31, 34, 35, 39]

print("Building Base Graphs (Unweighted Edges)...")
print("="*50)
print(f"Building graphs only for required timesteps: {REQUIRED_TIMESTEPS}")

# Build only the graphs we need for efficiency
base_graphs = {}
for timestep in tqdm(REQUIRED_TIMESTEPS, desc="Building base graphs"):
    graphs_for_timestep = build_emergence_graphs_for_time_range(
        edges_with_labels_df=edges_with_edge_labels,
        nodes_with_classes_df=nodes_with_labels,
        first_time_step=timestep,
        last_time_step=timestep,  # Build only this timestep
        max_walk_length=MAX_WALK_LENGTH,
        time_horizon=TIME_HORIZON,
        use_distance_labels=USE_DISTANCE_LABELS,
        keep_class_labels_as_features=False,
        ignore_illict=True,
        ignore_previously_transacting_with_illicit=True
    )
    base_graphs[timestep] = graphs_for_timestep[0]  # Should be only one graph

print(f"\nBuilt {len(base_graphs)} base graphs for timesteps: {list(base_graphs.keys())}")
sample_timestep = REQUIRED_TIMESTEPS[0]
sample_graph = base_graphs[sample_timestep]
print(f"Example base graph (t={sample_timestep}): {sample_graph.num_nodes} nodes, {sample_graph.edge_index.shape[1]} edges")
print(f"Base graph edge_attr: {hasattr(sample_graph, 'edge_attr')}")  # Should be False

Building Base Graphs (Unweighted Edges)...
Building graphs only for required timesteps: [20, 23, 30, 31, 34, 35, 39]


Building base graphs:   0%|          | 0/7 [00:00<?, ?it/s]

Total unique addresses across all time: 822942
Total time steps: 49
Generating 1 graphs (time steps 20 to 20)...

Pre-processing edges by time step...
Pre-processing edges by time step...

Building graphs...


Building base graphs:  14%|█▍        | 1/7 [00:33<03:19, 33.24s/it]

  t=20: nodes=345524, edges=1084995, labels={0: 299479, 1: 46045}

Stored 1 graphs
Total unique addresses across all time: 822942
Total time steps: 49
Generating 1 graphs (time steps 23 to 23)...

Pre-processing edges by time step...
Pre-processing edges by time step...

Building graphs...

Building graphs...


...

  t=34: nodes=530840, edges=1810073, labels={0: 474089, 1: 56751}

Stored 1 graphs
Total unique addresses across all time: 822942
Total time steps: 49
Generating 1 graphs (time steps 35 to 35)...

Pre-processing edges by time step...
Pre-processing edges by time step...

Building graphs...

Building graphs...


Building base graphs:  86%|████████▌ | 6/7 [05:14<00:56, 56.53s/it]

  t=35: nodes=552376, edges=1860027, labels={0: 504824, 1: 47552}

Stored 1 graphs
Total unique addresses across all time: 822942
Total time steps: 49
Generating 1 graphs (time steps 39 to 39)...

Pre-processing edges by time step...
Pre-processing edges by time step...

Building graphs...

Building graphs...


Building base graphs: 100%|██████████| 7/7 [05:54<00:00, 50.62s/it]

  t=39: nodes=620835, edges=2011116, labels={0: 567500, 1: 53335}

Stored 1 graphs

Built 7 base graphs for timesteps: [20, 23, 30, 31, 34, 35, 39]
Example base graph (t=20): 345524 nodes, 1084995 edges
Base graph edge_attr: True


In [12]:
# Build edge-weighted graphs using OPTIMIZED implementation
print("\nBuilding Edge-Weighted Graphs (OPTIMIZED)...")
print("="*50)
print(f"Building edge-weighted graphs for timesteps: {REQUIRED_TIMESTEPS}")

# Import the optimized functions
import sys
sys.path.append('../')
from optimized_edge_weights import compute_temporal_edge_weights_with_defaults_optimized

# Build graphs with temporal edge weights for required timesteps only
edge_weighted_graphs = {}

# Temporal edge weight hyperparameters
DECAY_LAMBDA = 0.05  # Recommended default
TEMPERATURE_TAU = 1.0  # Recommended default

for timestep in tqdm(REQUIRED_TIMESTEPS, desc="Building weighted graphs"):
    try:
        import time
        start_time = time.time()
        
        # Get data up to current time step
        nodes_up_to_t = nodes_with_labels[nodes_with_labels['Time step'] <= timestep]
        edges_up_to_t = edges_with_edge_labels[edges_with_edge_labels['Time step'] <= timestep]
        
        # Use all addresses that appear in nodes up to t
        active_addresses = nodes_up_to_t['address'].unique()
        address_to_local_id = {addr: idx for idx, addr in enumerate(active_addresses)}
        
        print(f"\\n  Timestep {timestep}: {len(edges_up_to_t)} edges, {len(active_addresses)} nodes")
        
        # Extract node features (without class labels to avoid leakage)
        node_features = extract_node_features(
            nodes_up_to_t, active_addresses, address_to_local_id,
            keep_class_labels_as_features=False, 
            current_time_step=timestep
        )
        
        # Build temporal edge weights using OPTIMIZED function
        edge_weight_start = time.time()
        edge_index, edge_weights = compute_temporal_edge_weights_with_defaults_optimized(
            edges_up_to_t, address_to_local_id, timestep,
            decay_lambda=DECAY_LAMBDA, temperature_tau=TEMPERATURE_TAU,
            value_column='total_BTC', timestep_column='Time step'
        )
        edge_weight_time = time.time() - edge_weight_start
        
        print(f"    Edge weighting took: {edge_weight_time:.2f}s")
        
        # Get node classes for reference
        node_classes = extract_node_classes(active_addresses, address_to_local_id, nodes_with_labels)
        
        # Use the same labels as base graphs for fair comparison
        base_graph_at_t = base_graphs[timestep]
        labels = base_graph_at_t.y
        
        # Create graph with edge weights
        weighted_graph = Data(
            x=node_features,
            edge_index=edge_index,
            edge_attr=edge_weights.unsqueeze(1),  # Add feature dimension
            y=labels,
            node_class=node_classes,
            num_nodes=len(active_addresses),
            time_step=timestep
        )
        
        edge_weighted_graphs[timestep] = weighted_graph
        
        total_time = time.time() - start_time
        print(f"    Total time for timestep {timestep}: {total_time:.2f}s")
        
    except Exception as e:
        print(f"Error building weighted graph at t={timestep}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\\nBuilt {len(edge_weighted_graphs)} edge-weighted graphs for timesteps: {list(edge_weighted_graphs.keys())}")
if edge_weighted_graphs:
    sample_timestep = REQUIRED_TIMESTEPS[0]
    sample_graph = edge_weighted_graphs[sample_timestep]
    print(f"Example weighted graph (t={sample_timestep}): {sample_graph.num_nodes} nodes, {sample_graph.edge_index.shape[1]} edges")
    print(f"Edge weights shape: {sample_graph.edge_attr.shape}")
    print(f"Sample edge weights: {sample_graph.edge_attr[:5].flatten()}")
    
    # Verify softmax normalization
    edge_index = sample_graph.edge_index
    edge_weights = sample_graph.edge_attr.flatten()
    print(f"\\nEdge weight statistics:")
    print(f"  Min: {edge_weights.min():.6f}, Max: {edge_weights.max():.6f}")
    print(f"  Mean: {edge_weights.mean():.6f}, Std: {edge_weights.std():.6f}")
    
    # Check softmax property (weights sum to 1 per destination)
    unique_dests = torch.unique(edge_index[1])
    sample_dest = unique_dests[0] if len(unique_dests) > 0 else None
    if sample_dest is not None:
        dest_mask = edge_index[1] == sample_dest
        dest_weight_sum = edge_weights[dest_mask].sum()
        print(f"  Sample destination node {sample_dest}: incoming weights sum = {dest_weight_sum:.6f}")
        
print("\\nOptimized graph building completed ✓")
print("\\n💡 Performance improvements:")
print("   - Vectorized operations instead of row-wise apply")
print("   - NumPy aggregations instead of pandas groupby")
print("   - Efficient softmax computation")
print("   - Expected speedup: 10-100x faster")


Building Edge-Weighted Graphs (OPTIMIZED)...
Building edge-weighted graphs for timesteps: [20, 23, 30, 31, 34, 35, 39]


Building weighted graphs:   0%|          | 0/7 [00:00<?, ?it/s]


Building Edge-Weighted Graphs (OPTIMIZED)...
Building edge-weighted graphs for timesteps: [20, 23, 30, 31, 34, 35, 39]


Building weighted graphs:   0%|          | 0/7 [00:00<?, ?it/s]

\n  Timestep 20: 1084995 edges, 345524 nodes
    Edge weighting took: 5.00s
    Edge weighting took: 5.00s


...

    Total time for timestep 34: 18.56s
\n  Timestep 35: 1860027 edges, 552376 nodes
\n  Timestep 35: 1860027 edges, 552376 nodes
    Edge weighting took: 8.74s
    Edge weighting took: 8.74s


Building weighted graphs:  86%|████████▌ | 6/7 [01:37<00:17, 17.43s/it]

    Total time for timestep 35: 19.05s
\n  Timestep 39: 2011116 edges, 620835 nodes
\n  Timestep 39: 2011116 edges, 620835 nodes
    Edge weighting took: 9.66s
    Edge weighting took: 9.66s


Building weighted graphs: 100%|██████████| 7/7 [01:58<00:00, 16.94s/it]

    Total time for timestep 39: 21.47s
\nBuilt 7 edge-weighted graphs for timesteps: [20, 23, 30, 31, 34, 35, 39]
Example weighted graph (t=20): 345524 nodes, 1044294 edges
Edge weights shape: torch.Size([1044294, 1])
Sample edge weights: tensor([1., 1., 1., 1., 1.])
\nEdge weight statistics:
  Min: 0.000000, Max: 1.000000
  Mean: 0.261397, Std: 0.337248
  Sample destination node 0: incoming weights sum = 1.000000
\nOptimized graph building completed ✓
\n💡 Performance improvements:
   - Vectorized operations instead of row-wise apply
   - NumPy aggregations instead of pandas groupby
   - Efficient softmax computation
   - Expected speedup: 10-100x faster


In [ ]:
# Quick benchmark: Original vs Optimized Edge Weighting
print("\\nPerformance Benchmark: Original vs Optimized")
print("="*60)

# Test with a smaller subset first
test_timestep = 20
test_edges = edges_with_edge_labels[edges_with_edge_labels['Time step'] <= test_timestep]
test_nodes = nodes_with_labels[nodes_with_labels['Time step'] <= test_timestep]
test_addresses = test_nodes['address'].unique()
test_address_map = {addr: idx for idx, addr in enumerate(test_addresses)}

print(f"Test data: {len(test_edges)} edges, {len(test_addresses)} addresses")

# Import both versions
from optimized_edge_weights import compute_temporal_edge_weights_with_defaults_optimized

import time

# Benchmark optimized version
print("\\nTesting optimized version...")
start = time.time()
edge_index_opt, weights_opt = compute_temporal_edge_weights_with_defaults_optimized(
    test_edges, test_address_map, test_timestep,
    value_column='total_BTC', timestep_column='Time step'
)
opt_time = time.time() - start

# Benchmark original version 
print("Testing original version...")
start = time.time()
try:
    edge_index_orig, weights_orig = compute_temporal_edge_weights_with_defaults(
        test_edges, test_address_map, test_timestep,
        value_column='total_BTC', timestep_column='Time step'
    )
    orig_time = time.time() - start
    
    # Calculate speedup
    speedup = orig_time / opt_time
    print(f"\\n📊 PERFORMANCE RESULTS:")
    print(f"   Original: {orig_time:.2f}s")
    print(f"   Optimized: {opt_time:.2f}s") 
    print(f"   Speedup: {speedup:.1f}x faster 🚀")
    
    # Verify consistency
    if edge_index_orig.shape == edge_index_opt.shape:
        print(f"   ✅ Results consistent: {edge_index_opt.shape[1]} edges")
    else:
        print(f"   ⚠️ Edge counts differ: {edge_index_orig.shape[1]} vs {edge_index_opt.shape[1]}")
        
except Exception as e:
    print(f"   Original version failed: {e}")
    print(f"   Optimized version: {opt_time:.2f}s ✅")
    
print(f"\\nProceeding with optimized implementation...")

\nPerformance Benchmark: Original vs Optimized
Test data: 1084995 edges, 345524 addresses
\nTesting optimized version...
Test data: 1084995 edges, 345524 addresses
\nTesting optimized version...
Testing original version...
Testing original version...


In [ ]:
# Create train/test splits for both graph types
print("Creating train/test splits...")

# Same timesteps as original baseline (indices [19, 22, 29, 30] and [33, 34, 38])
# These correspond to timesteps [20, 23, 30, 31] and [34, 35, 39]
train_timesteps = [20, 23, 30, 31]
test_timesteps = [34, 35, 39]

print(f"Train timesteps: {train_timesteps}")
print(f"Test timesteps: {test_timesteps}")

# Base graphs splits
base_train_graphs = [base_graphs[t] for t in train_timesteps if t in base_graphs]
base_test_graphs = [base_graphs[t] for t in test_timesteps if t in base_graphs]

# Edge-weighted graphs splits 
edge_train_graphs = [edge_weighted_graphs[t] for t in train_timesteps if t in edge_weighted_graphs]
edge_test_graphs = [edge_weighted_graphs[t] for t in test_timesteps if t in edge_weighted_graphs]

print(f"Base graphs - Train: {len(base_train_graphs)}, Test: {len(base_test_graphs)}")
print(f"Edge-weighted graphs - Train: {len(edge_train_graphs)}, Test: {len(edge_test_graphs)}")

# Verify we have matching numbers for fair comparison
min_train = min(len(base_train_graphs), len(edge_train_graphs))
min_test = min(len(base_test_graphs), len(edge_test_graphs))

base_train_graphs = base_train_graphs[:min_train]
base_test_graphs = base_test_graphs[:min_test]
edge_train_graphs = edge_train_graphs[:min_train]
edge_test_graphs = edge_test_graphs[:min_test]

print(f"Final matched splits - Train: {min_train}, Test: {min_test}")

# Print label distribution for each split
for split_name, graphs in [("Base Train", base_train_graphs), ("Base Test", base_test_graphs),
                           ("Edge Train", edge_train_graphs), ("Edge Test", edge_test_graphs)]:
    if graphs:  # Check if we have graphs
        all_labels = torch.cat([graph.y for graph in graphs])
        label_counts = Counter(all_labels.numpy())
        total = len(all_labels)
        print(f"\n{split_name} label distribution:")
        print(f"  Class 0 (No emergence): {label_counts[0]:,} ({100*label_counts[0]/total:.1f}%)")
        print(f"  Class 1 (Emergence): {label_counts[1]:,} ({100*label_counts[1]/total:.1f}%)")
        print(f"  Total: {total:,} nodes")
    else:
        print(f"\n{split_name}: No graphs available")

## 4. Define GCN Model Architecture

In [ ]:
class EdgeWeightedGCN(torch.nn.Module):
    """
    GCN model that supports both unweighted and edge-weighted message passing.
    
    For unweighted graphs: Uses standard binary adjacency
    For edge-weighted graphs: Uses edge weights in message passing
    """
    
    def __init__(self, num_features, hidden_dim, num_classes, dropout=0.2):
        super(EdgeWeightedGCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.classifier = torch.nn.Linear(hidden_dim, num_classes)
        self.dropout = dropout
        
    def forward(self, x, edge_index, edge_weight=None):
        """
        Forward pass with optional edge weights.
        
        Args:
            x: Node features [num_nodes, num_features]
            edge_index: Edge connectivity [2, num_edges]
            edge_weight: Optional edge weights [num_edges] or [num_edges, 1]
        """
        
        # Handle edge weights - flatten if needed
        if edge_weight is not None:
            if edge_weight.dim() > 1:
                edge_weight = edge_weight.flatten()
        
        # First GCN layer
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Second GCN layer
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Classifier
        x = self.classifier(x)
        return x

# Model hyperparameters (same as baseline for fair comparison)
HIDDEN_DIM = 64
NUM_EPOCHS = 50  # Increased for better convergence
LEARNING_RATE = 0.01
WEIGHT_DECAY = 5e-4

# Get input dimensions from data
if base_train_graphs:
    num_features = base_train_graphs[0].x.shape[1]
    num_classes = 2
    print(f"Model configuration:")
    print(f"  Input features: {num_features}")
    print(f"  Hidden dimension: {HIDDEN_DIM}")
    print(f"  Output classes: {num_classes}")
    print(f"  Dropout: 0.2")
else:
    print("No training graphs available!")

## 5. Training Functions for Both Graph Types

In [ ]:
def train_epoch(model, graphs, optimizer, use_edge_weights=False):
    """
    Train model for one epoch on a list of graphs.
    
    Args:
        model: The GCN model
        graphs: List of graph objects
        optimizer: PyTorch optimizer
        use_edge_weights: Whether to use edge weights from graph.edge_attr
    """
    model.train()
    total_loss = 0
    total_correct = 0
    total_nodes = 0
    
    for graph in graphs:
        graph = graph.to(DEVICE)
        
        optimizer.zero_grad()
        
        # Get edge weights if requested and available
        edge_weight = None
        if use_edge_weights and hasattr(graph, 'edge_attr') and graph.edge_attr is not None:
            edge_weight = graph.edge_attr
        
        # Forward pass
        out = model(graph.x, graph.edge_index, edge_weight)
        loss = F.cross_entropy(out, graph.y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        pred = out.argmax(dim=1)
        correct = (pred == graph.y).sum().item()
        
        total_loss += loss.item() * graph.num_nodes
        total_correct += correct
        total_nodes += graph.num_nodes
    
    avg_loss = total_loss / total_nodes
    avg_acc = total_correct / total_nodes
    return avg_loss, avg_acc


def evaluate(model, graphs, use_edge_weights=False):
    """
    Evaluate model on a list of graphs.
    
    Args:
        model: The GCN model
        graphs: List of graph objects
        use_edge_weights: Whether to use edge weights from graph.edge_attr
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for graph in graphs:
            graph = graph.to(DEVICE)
            
            # Get edge weights if requested and available
            edge_weight = None
            if use_edge_weights and hasattr(graph, 'edge_attr') and graph.edge_attr is not None:
                edge_weight = graph.edge_attr
            
            # Forward pass
            out = model(graph.x, graph.edge_index, edge_weight)
            probs = F.softmax(out, dim=1)
            preds = out.argmax(dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(graph.y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels), np.array(all_probs)


def train_model(model_name, train_graphs, test_graphs, use_edge_weights=False, 
                num_epochs=NUM_EPOCHS, verbose=True):
    """
    Train a complete model and return training history.
    
    Args:
        model_name: String name for the model (for logging)
        train_graphs: Training graphs
        test_graphs: Test graphs
        use_edge_weights: Whether to use edge weights
        num_epochs: Number of training epochs
        verbose: Whether to print progress
    """
    
    # Create fresh model and optimizer
    model = EdgeWeightedGCN(num_features, HIDDEN_DIM, num_classes).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_acc': [],
        'epochs': []
    }
    
    if verbose:
        print(f"\nTraining {model_name}")
        print("="*50)
    
    # Training loop
    progress_bar = tqdm(range(1, num_epochs + 1), desc=f"{model_name} Training") if verbose else range(1, num_epochs + 1)
    
    for epoch in progress_bar:
        # Train
        train_loss, train_acc = train_epoch(model, train_graphs, optimizer, use_edge_weights)
        
        # Evaluate periodically
        test_acc = None
        if epoch % 10 == 0 or epoch == 1 or epoch == num_epochs:
            test_preds, test_labels, test_probs = evaluate(model, test_graphs, use_edge_weights)
            test_acc = (test_preds == test_labels).mean()
        
        # Record history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['epochs'].append(epoch)
        
        # Update progress
        if verbose:
            desc = f"{model_name} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}"
            if test_acc is not None:
                desc += f" | Test Acc: {test_acc:.4f}"
            progress_bar.set_description(desc)
    
    return model, history

print("Training functions defined ✓")

## 6. Train Base GCN Model

In [ ]:
# Train Base GCN Model
base_model, base_history = train_model(
    model_name="Base GCN",
    train_graphs=base_train_graphs,
    test_graphs=base_test_graphs,
    use_edge_weights=False,
    num_epochs=NUM_EPOCHS,
    verbose=True
)

In [ ]:
# Extract results for comparison
base_train_losses = base_history['train_loss']
base_test_accs = [acc for acc in base_history['test_acc'] if acc is not None]
best_base_acc = max(base_test_accs) if base_test_accs else 0.0

print(f"\nBase Model Training Complete!")
print(f"Best Test Accuracy: {best_base_acc:.4f}")
print(f"Final Training Loss: {base_train_losses[-1]:.4f}")
print(f"Total epochs: {len(base_train_losses)}")

# Detailed evaluation of Base GCN (like in train_gcn_baseline.ipynb)
print("\n" + "=" * 50)
print("BASE GCN - DETAILED EVALUATION")
print("=" * 50)

base_final_preds, base_final_labels, base_final_probs = evaluate(base_model, base_test_graphs, use_edge_weights=False)
print(classification_report(base_final_labels, base_final_preds, target_names=['No Emergence', 'Emergence']))

if len(np.unique(base_final_labels)) == 2:
    base_auc = roc_auc_score(base_final_labels, base_final_probs[:, 1])
    print(f"ROC-AUC: {base_auc:.4f}")

print("\nConfusion Matrix:")
base_cm = confusion_matrix(base_final_labels, base_final_preds)
print(base_cm)

## 7. Train Edge-Weighted GCN Model

In [ ]:
# Train Edge-Weighted GCN Model
edge_model, edge_history = train_model(
    model_name="Edge-Weighted GCN",
    train_graphs=edge_train_graphs,
    test_graphs=edge_test_graphs,
    use_edge_weights=True,
    num_epochs=NUM_EPOCHS,
    verbose=True
)

# Extract results for comparison
edge_train_losses = edge_history['train_loss']
edge_test_accs = [acc for acc in edge_history['test_acc'] if acc is not None]
best_edge_acc = max(edge_test_accs) if edge_test_accs else 0.0

print(f"\nEdge-Weighted Model Training Complete!")
print(f"Best Test Accuracy: {best_edge_acc:.4f}")
print(f"Final Training Loss: {edge_train_losses[-1]:.4f}")
print(f"Total epochs: {len(edge_train_losses)}")

## 8. Compare Results

In [ ]:
# Comprehensive model comparison
print("Model Comparison Results")
print("=" * 60)

# Performance comparison
print(f"📊 PERFORMANCE METRICS")
print(f"{'Metric':<25} {'Base GCN':<15} {'Edge-Weighted':<15} {'Improvement':<15}")
print("-" * 70)
print(f"{'Best Test Accuracy':<25} {best_base_acc:<15.4f} {best_edge_acc:<15.4f} {((best_edge_acc - best_base_acc) / best_base_acc * 100):+.2f}%")
print(f"{'Final Train Loss':<25} {base_train_losses[-1]:<15.4f} {edge_train_losses[-1]:<15.4f} {((edge_train_losses[-1] - base_train_losses[-1]) / base_train_losses[-1] * 100):+.2f}%")
print(f"{'Training Epochs':<25} {len(base_train_losses):<15} {len(edge_train_losses):<15}")

# Statistical significance test (simple t-test on test accuracies)
from scipy import stats
if len(base_test_accs) >= 3 and len(edge_test_accs) >= 3:
    t_stat, p_value = stats.ttest_ind(base_test_accs, edge_test_accs)
    print(f"{'P-value (t-test)':<25} {p_value:<15.4f} {'Significant' if p_value < 0.05 else 'Not significant'}")

print(f"\n🏆 WINNER: {'Edge-Weighted GCN' if best_edge_acc > best_base_acc else 'Base GCN'}")

# Model complexity comparison
print(f"\n📈 MODEL COMPLEXITY")
base_params = sum(p.numel() for p in base_model.parameters())
edge_params = sum(p.numel() for p in edge_model.parameters())
sample_base_graph = base_train_graphs[0]
sample_edge_graph = edge_train_graphs[0]
print(f"{'Model':<25} {'Parameters':<15} {'Graph Edges':<15} {'Edge Features':<15}")
print("-" * 70)
print(f"{'Base GCN':<25} {base_params:<15} {sample_base_graph.edge_index.shape[1]:<15} {'None':<15}")
print(f"{'Edge-Weighted GCN':<25} {edge_params:<15} {sample_edge_graph.edge_index.shape[1]:<15} {'Temporal weights':<15}")

# Training stability analysis
print(f"\n📉 TRAINING STABILITY")
base_acc_std = np.std(base_test_accs) if len(base_test_accs) > 1 else 0.0
edge_acc_std = np.std(edge_test_accs) if len(edge_test_accs) > 1 else 0.0
print(f"{'Model':<25} {'Test Acc Std':<20} {'Convergence':<15}")
print("-" * 60)
print(f"{'Base GCN':<25} {base_acc_std:<20.4f} {'Stable' if base_acc_std < 0.01 else 'Unstable'}")
print(f"{'Edge-Weighted GCN':<25} {edge_acc_std:<20.4f} {'Stable' if edge_acc_std < 0.01 else 'Unstable'}")

# Summary insights
print(f"\n💡 KEY INSIGHTS")
improvement = (best_edge_acc - best_base_acc) / best_base_acc * 100
if improvement > 2:
    print(f"✅ Edge weighting provides significant improvement ({improvement:.2f}%)")
    print("✅ Temporal information enhances GCN performance for transaction classification")
elif improvement > 0:
    print(f"⚠️  Edge weighting provides modest improvement ({improvement:.2f}%)")
    print("⚠️  Temporal information has limited benefit for this task")
else:
    print(f"❌ Edge weighting decreases performance ({improvement:.2f}%)")
    print("❌ Temporal information may be adding noise or the decay parameters need tuning")

if edge_acc_std < base_acc_std:
    print("✅ Edge weighting improves training stability")
else:
    print("⚠️  Edge weighting may reduce training stability")

## 9. Visualize Training Progress and Results

In [ ]:
import matplotlib.pyplot as plt

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Base GCN vs Edge-Weighted GCN Comparison', fontsize=16, fontweight='bold')

# 1. Training Loss Comparison
axes[0, 0].plot(base_train_losses, label='Base GCN', linewidth=2, alpha=0.8)
axes[0, 0].plot(edge_train_losses, label='Edge-Weighted GCN', linewidth=2, alpha=0.8)
axes[0, 0].set_title('Training Loss Over Time')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Training Accuracy Comparison
base_train_accs = base_history['train_acc']
edge_train_accs = edge_history['train_acc']
axes[0, 1].plot(base_train_accs, label='Base GCN', linewidth=2, alpha=0.8)
axes[0, 1].plot(edge_train_accs, label='Edge-Weighted GCN', linewidth=2, alpha=0.8)
axes[0, 1].set_title('Training Accuracy Over Time')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Performance Bar Chart
models = ['Base GCN', 'Edge-Weighted GCN']
accs = [best_base_acc, best_edge_acc]
colors = ['lightblue', 'lightcoral']
bars = axes[0, 2].bar(models, accs, color=colors, alpha=0.7, edgecolor='black')
axes[0, 2].set_title('Best Test Accuracy Comparison')
axes[0, 2].set_ylabel('Accuracy')
axes[0, 2].set_ylim(min(accs) - 0.02, max(accs) + 0.02)
# Add value labels on bars
for bar, acc in zip(bars, accs):
    axes[0, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                    f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

# 4. Edge Weight Distribution
if edge_train_graphs:
    sample_edge_graph = edge_train_graphs[0]
    axes[1, 0].hist(sample_edge_graph.edge_attr.cpu().numpy(), bins=50, alpha=0.7, color='orange', edgecolor='black')
    axes[1, 0].set_title('Edge Weight Distribution')
    axes[1, 0].set_xlabel('Edge Weight')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No edge-weighted graphs', ha='center', va='center', transform=axes[1, 0].transAxes)

# 5. Test Accuracy Comparison (with evaluation points)
base_test_epochs = [i for i, acc in enumerate(base_history['test_acc']) if acc is not None]
base_test_values = [acc for acc in base_history['test_acc'] if acc is not None]
edge_test_epochs = [i for i, acc in enumerate(edge_history['test_acc']) if acc is not None]
edge_test_values = [acc for acc in edge_history['test_acc'] if acc is not None]

if base_test_values and edge_test_values:
    axes[1, 1].plot(base_test_epochs, base_test_values, 'o-', label='Base GCN', linewidth=2, markersize=6)
    axes[1, 1].plot(edge_test_epochs, edge_test_values, 's-', label='Edge-Weighted GCN', linewidth=2, markersize=6)
    axes[1, 1].set_title('Test Accuracy at Evaluation Points')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Test Accuracy')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

# 6. Training Loss Smoothed
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

base_smooth = smooth_curve(base_train_losses)
edge_smooth = smooth_curve(edge_train_losses)

axes[1, 2].plot(base_train_losses, alpha=0.3, color='blue', label='Base (raw)')
axes[1, 2].plot(base_smooth, color='blue', linewidth=2, label='Base (smoothed)')
axes[1, 2].plot(edge_train_losses, alpha=0.3, color='red', label='Edge-weighted (raw)')
axes[1, 2].plot(edge_smooth, color='red', linewidth=2, label='Edge-weighted (smoothed)')
axes[1, 2].set_title('Smoothed Training Loss')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('Loss')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics table
print("\n📊 FINAL SUMMARY TABLE")
print("=" * 80)
summary_data = {
    'Metric': ['Best Test Acc', 'Final Train Loss', 'Training Epochs', 'Test Acc Std', 'Parameters'],
    'Base GCN': [f"{best_base_acc:.4f}", f"{base_train_losses[-1]:.4f}", 
                 len(base_train_losses), f"{base_acc_std:.4f}", f"{base_params:,}"],
    'Edge-Weighted': [f"{best_edge_acc:.4f}", f"{edge_train_losses[-1]:.4f}", 
                      len(edge_train_losses), f"{edge_acc_std:.4f}", f"{edge_params:,}"],
    'Improvement': [f"{((best_edge_acc - best_base_acc) / best_base_acc * 100):+.2f}%",
                    f"{((edge_train_losses[-1] - base_train_losses[-1]) / base_train_losses[-1] * 100):+.2f}%",
                    f"{len(edge_train_losses) - len(base_train_losses):+d}",
                    f"{((edge_acc_std - base_acc_std) / max(base_acc_std, 1e-6) * 100):+.2f}%",
                    "Same"]
}

for i, metric in enumerate(summary_data['Metric']):
    print(f"{metric:<15} | {summary_data['Base GCN'][i]:<12} | {summary_data['Edge-Weighted'][i]:<12} | {summary_data['Improvement'][i]:<12}")

print("\n🎯 CONCLUSION:")
if best_edge_acc > best_base_acc:
    improvement = (best_edge_acc - best_base_acc) / best_base_acc * 100
    print(f"✅ Edge-weighted GCN outperforms base GCN by {improvement:.2f}%")
    print("✅ Temporal edge weighting successfully captures important patterns in transaction data")
else:
    decline = (best_base_acc - best_edge_acc) / best_base_acc * 100
    print(f"❌ Base GCN outperforms edge-weighted GCN by {decline:.2f}%")
    print("❌ Current temporal weighting parameters may need tuning or temporal info is not beneficial")